In [ ]:
!pip install pandas
!pip install geopandas
!pip install plotly
!pip install dash
!pip install dash_bootstrap_components

In [ ]:
import pandas as pd
import geopandas as gpd
import json

import plotly
import plotly.express as px

import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc


In [ ]:
data = pd.read_excel("entgelte_data.xlsx", engine="openpyxl")

In [ ]:
data

In [ ]:
data.replace('k.A.', None, inplace = True)
data[data['Jahr'] == 2017]

In [ ]:
data['Durchschnittliches monatliches Arbeitsentgelt'].sum()

In [ ]:
# wir ändern Durchschnittliches monatliches Arbeitsentgelt in einen griffigeren Spaltennamen
data.columns = data.columns.str.replace('Durchschnittliches monatliches Arbeitsentgelt', 'Monatsgehalt')


In [ ]:
# wir checken, ob alle Bundesländer richtig geschrieben wurden
data['Bundesland'].unique()


In [ ]:
# Jedem Element der Spalte "Bundesland", bei dem am Ende von "Mecklenburg-Vorpommern" ein "n" fehlt, wird ein "n" angehängt

for item in data['Bundesland']:
    if item == 'Mecklenburg-Vorpommer':
        data.replace(item, str(item) + "n", inplace = True)


In [ ]:
# wir möchten die Zeilen mit dem Eintrag 'Bundesrepublik' entfernen, weil der Wert selbst berechnet werden kann
# Definition zeigt, an welchen Stellen im Index ein bestimmter Wert steht

def get_bl_index(bundesland):
    i = data.index
    index = data["Bundesland"] == bundesland
    result = i[index]
    bl_indices = result.tolist()
    return bl_indices


In [ ]:
#hier bekommen wir ausgegeben, an welchen Stellen Bundesrepublik im Index steht

get_bl_index('Bundesrepublik')


In [ ]:
#hier entfernen wir alle Zeilen mit Bundesrepublik

data.drop(get_bl_index('Bundesrepublik'), axis=0, inplace = True)


In [ ]:
data.to_csv(r'projekt_entgelte_final.csv', index=False)

In [ ]:
# Einlesen des Datensatzes mit den Arbeitsentgelten aus den Bundesländern
entgelte = pd.read_csv("projekt_entgelte_final.csv")

In [ ]:
# Abspeichern des Shapefiles für die deutschen Bundesländer
geodaten = "gadm41_DEU_1.shp"
#Einlesen der Geodaten mit geopandas
map_df = gpd.read_file(geodaten)
# Aussortieren der nicht benötigten Spalten aus dem Datensatz
map_df.drop(["GID_1", "GID_0", "COUNTRY", "VARNAME_1", "NL_NAME_1", "TYPE_1", "ENGTYPE_1", "CC_1", "HASC_1", "ISO_1"] , axis=1, inplace=True)

In [ ]:
# Vereinfachen der Polygone, aus denen sich die BUndesländer aufbauen, auf eine Genauigkeit von 1000 Metern
# So verringert sich die Zeit, die die Karte zum Laden braucht, deutlich
map_df["geometry"] = (map_df.to_crs(map_df.estimate_utm_crs()).simplify(1000).to_crs(map_df.crs))

In [ ]:
# Exportieren der Geodaten als GeoJSON
map_df.to_file("bundeslaender_shapefile.json", driver='GeoJSON')

In [ ]:
# Einlesen der erstellen GeoJSON-Datei
with open('bundeslaender_shapefile.json', encoding='utf-8') as shapefile:
    bundeslaender_geo = json.load(shapefile)

In [ ]:
# Abfrage, über welche keys auf die Daten im Shapefile zugegriffen werden kann
bundeslaender_geo["features"][0]["properties"].keys()


In [ ]:
# Visualisierung des Datensatzes als Choroplethen-Karte mit plotly.express

karte_entgelte = px.choropleth_mapbox(entgelte, geojson=bundeslaender_geo,
                          width=600, height=600,
                          locations='Bundesland',
                          title = "Durchschnittliche Monatsgehälter in WfbM 2009 - 2021 in Euro",
                          color='Monatsgehalt',
                          color_continuous_scale="sunset",
                          featureidkey="properties.NAME_1",
                          mapbox_style="carto-positron",
                          opacity=0.8,
                          center = {"lat": 51.1657, "lon": 10.4515},
                          zoom=4.3,
                          hover_name="Bundesland",
                          custom_data=['Bundesland', 'Monatsgehalt'],
                          animation_frame='Jahr'
                          )

karte_entgelte.update_geos(fitbounds="locations",visible=False)
karte_entgelte.update_layout(height=600, margin={"r":0,"t":70,"l":0,"b":0})
karte_entgelte.update_coloraxes(cmin=110, cmax=270)
karte_entgelte.update_traces(hovertemplate="<br>".join(["<b>%{customdata[0]}<b>",
                                             "Monatsgehalt: %{customdata[1]}" "€",
                                            ])
                 )

karte_entgelte.show()


In [ ]:
#Bundesdurchschnitt zum Vergleich der Entwicklung mit Sachsen
bundesdurchschnitt = entgelte.groupby('Jahr', as_index = False).mean()
bundesdurchschnitt


In [ ]:
#nur Sachsen extrahieren
data_sachsen = entgelte[entgelte['Bundesland']=='Sachsen']
data_sachsen


In [ ]:
#Bundesdurchschnitt und Sachsen zusammenfügen
vergleich_data = pd.concat([bundesdurchschnitt, data_sachsen])
vergleich_data


In [ ]:
#für Bundesdurchschnitt einen Wert vergeben für die Spalte Bundesland
vergleich_data['Bundesland'] = vergleich_data['Bundesland'].fillna('Bundesdurchschnitt')
vergleich_data


In [ ]:
#Darstellung der Entwicklung in Deutschland vs. Sachsen
import plotly.express as px

graph_entgelte = px.line(vergleich_data, x="Jahr", y="Monatsgehalt", title='Entwicklung der Entgelte in Euro', color='Bundesland')

graph_entgelte.show()


In [ ]:
data_anzahl_wfbm = pd.read_excel("werkstätten_data.xlsx", engine="openpyxl")
data_anzahl_wfbm

In [ ]:
# Datenbereinigung: Überprüfung, ob alle Bundesländer und Jahre richtig da sind
len(data_anzahl_wfbm['Bundesland'].unique())

In [ ]:
data_anzahl_wfbm['Bundesland'].unique()

In [ ]:
data_anzahl_wfbm['Jahr'].unique()

In [ ]:
len(data_anzahl_wfbm['Jahr'].unique())

In [ ]:
# Überprüfen, wie die Daten vorliegen und ob von allen Datentypen
# gleich viele Werte vorliegen
data_anzahl_wfbm.info()

In [ ]:
data_anzahl_wfbm

In [ ]:
bundesdurchschnitt_wfbm = data_anzahl_wfbm.groupby('Jahr', as_index = False)['Anzahl der amtlich anerkannten WfbM'].mean().round(0)
bundesdurchschnitt_wfbm

In [ ]:
bundesdurchschnitt_wfbm['Region'] = 'Deutschland (Durchschnitt)'
bundesdurchschnitt_wfbm

In [ ]:
Bund_Gesamtentwicklung=data_anzahl_wfbm.groupby('Jahr', as_index = False)['Anzahl der amtlich anerkannten WfbM'].sum()
Bund_Gesamtentwicklung

In [ ]:
Bund_Gesamtentwicklung['Region'] = 'Deutschland'
Bund_Gesamtentwicklung

In [ ]:
data_sachsen_wfbm = data_anzahl_wfbm[data_anzahl_wfbm['Bundesland']=='Sachsen']
data_sachsen_wfbm = data_sachsen_wfbm.groupby('Jahr', as_index = False)['Anzahl der amtlich anerkannten WfbM'].sum()
data_sachsen_wfbm

In [ ]:
data_sachsen_wfbm['Region'] = 'Sachsen'
data_sachsen_wfbm

In [ ]:
vergleich_werkstaetten = pd.concat([bundesdurchschnitt_wfbm, data_sachsen_wfbm])
vergleich_werkstaetten

In [ ]:
graph_werkstätten = px.line(vergleich_werkstaetten, x="Jahr", y="Anzahl der amtlich anerkannten WfbM", title='Entwicklung Anzahl der Betriebsstätten', color='Region')

graph_werkstätten.show()

In [ ]:
vergleich_werkstaetten2 = pd.concat([Bund_Gesamtentwicklung, data_sachsen_wfbm])
vergleich_werkstaetten2

In [ ]:
graph_werkstätten2 = px.line(vergleich_werkstaetten2, x="Jahr", y="Anzahl der amtlich anerkannten WfbM", title='Entwicklung Anzahl der Werkstätten (absolute Zahlen)', color='Region')

graph_werkstätten2.show()

In [ ]:
data_plätze = pd.read_csv("werkstattplätze_data.csv") 

In [ ]:
bundesdurchschnitt_plätze = data_plätze.groupby('Jahr', as_index = False)['Summe bel. Plätze'].mean().round(0)
bundesdurchschnitt_plätze['Region'] = 'Deutschland (Durchschnitt)'
bundesdurchschnitt_plätze

In [ ]:
data_sachsen_plätze = data_plätze[data_plätze['Bundesland']=='Sachsen']
data_sachsen_plätze = data_sachsen_plätze.groupby('Jahr', as_index = False)['Summe bel. Plätze'].sum()
data_sachsen_plätze

In [ ]:
data_sachsen_plätze['Region'] = 'Sachsen'
data_sachsen_plätze

In [ ]:
vergleich_plätze = pd.concat([bundesdurchschnitt_plätze, data_sachsen_plätze])
vergleich_plätze

In [ ]:
graph_werkstattplätze_durchschnitt = px.line(vergleich_plätze, x="Jahr", y="Summe bel. Plätze", title='Entwicklung Anzahl belegter Plätze in Werkstätten', color='Region')

graph_werkstattplätze_durchschnitt.show()

In [ ]:
# Einlesen des Werkstatt-Datensatzes
werkstaetten = gpd.read_file("Karte_Werkstätten.geojson")

print(werkstaetten.head())

In [ ]:
# Fügt dem Werkstätten-Datensatz zwei neue Spalten hinzu, die die Latitude- und Longitude-Werte einzeln enthalten.
# Plotly benötigt die Geo-Angaben in dieser Form, um die Punkte auf der Karte korrekt darzustellen.
werkstaetten['lon'] = werkstaetten['geometry'].x
werkstaetten['lat'] = werkstaetten['geometry'].y

# Funktion, die die Namen der Werkstätten nach Leerzeichen durchsucht und beim jeweils fünften Leerzeichen ein <br>-Tag einbaut.
# Diese geänderte Version der Werkstatt-Namen wird anschließend in einer neuen Spalte im Werkstatt-Datensatz abgespeichert.
# Dieser Schritt ermöglicht es, bei der Visualisierung die Info-Box kleiner zu halten, die beim Hovern über die Punkte auftaucht,
# da die oft sehr langen Werkstatt-Namen ab einer gewissen Länge mit einem Absatz getrennt werden.
def add_br():
    names_formatted = []
    for werkstatt in werkstaetten["Name"]:
        name_neu = ""
        counter = 0
        for zeichen in range(len(werkstatt)):
            if werkstatt[zeichen] == ' ':
                counter += 1
            else:
                counter += 0
            if werkstatt[zeichen] == ' ' and counter == 5:
                name_neu = name_neu + '<br>'
            else:
                name_neu = name_neu + werkstatt[zeichen]
        names_formatted.append(name_neu)
    return names_formatted

werkstaetten['Name_formatted'] = pd.Series(add_br())

In [ ]:
# Festlegen verschiedener Parameter für die Visualisierung der Daten als Scatter Plot auf einer Karte.
# Die Grundlage stellt eine Mapbox von Carto dar, die frei verwendbar ist. Diese wird mit Hilfe von 'center' und 'zoom'
# auf die richtige Ansicht eingestellt.
# Für die Geo-Daten werden die Latitude- und Longitude-Werte aus dem Werkstatt-Datensatz verwendet.
karte_werkstätten = px.scatter_mapbox(werkstaetten, lat="lat", lon="lon",
                        width=900, height=500,
                        title="WfbM in Sachsen",
                        hover_name="Name_formatted",
                        labels={"Foerderung":"Besondere Fördermöglichkeiten"},
                        color="Foerderung",
                        color_discrete_sequence=['#bcbd22', '#17becf'],
                        mapbox_style="carto-positron",
                        opacity=1,
                        center = {"lat": 50.98525, "lon": 13.58035},
                        zoom=6.8,
                        # animation_frame="Foerderung"
                        )

# Anpassen der Größe der Punkte auf der Karte für bessere Sichtbarkeit
karte_werkstätten.update_traces(marker={'size': 12})

# Anpassen des Titels der Legende
karte_werkstätten.update_layout(legend=dict(title="Besondere<br>Fördermöglichkeiten"))

karte_werkstätten.show()

In [ ]:
pflichtarbeitsplätze = pd.read_excel('pflichtarbeitsplätze_data.xlsx', engine="openpyxl")

In [ ]:
pflichtarbeitsplätze

In [ ]:
pflichtarbeitsplätze["unbesetzt"].mean()

In [ ]:
#überprüfen, ob alle Zahlenangaben auch als integer oder floats erkannt werden, so dass damit gerechnet werden kann
print(pflichtarbeitsplätze.info())


In [ ]:
#Jahre 2009 bis 2020 werden abgebildet im Datensatz
pflichtarbeitsplätze['Jahr'].unique()


In [ ]:
#Überprüfung der Bundesländer, Überprüfung, dass Bundesländer immer gleich geschrieben werden
#es fällt auf: Sachsen-Anhalt ist falsch geschrieben
#überlegung: Bedeutung von Angabe fehlt, umbenennen in intuitiveren Namen
pflichtarbeitsplätze['Bundesland'].unique()


In [ ]:
len(pflichtarbeitsplätze['Bundesland'].unique())

In [ ]:
#falsche Bundesland-Einträge wurden ersetzt und durch inplace=True im Datensatz übernommen
pflichtarbeitsplätze.replace('Sachsten-Anhalt','Sachsen-Anhalt', inplace=True)
pflichtarbeitsplätze.replace('Nordrhein-Westfahlen','Nordrhein-Westfalen', inplace=True)


In [ ]:
#Kontrolle, dass neue Werte übernommen wurden
pflichtarbeitsplätze[pflichtarbeitsplätze['Bundesland'] == 'Sachsen-Anhalt']


In [ ]:
#'Angabe fehlt' wird ersetzt durch intutiveren Namen
pflichtarbeitsplätze.replace('Angabe fehlt','keinem Bundesland zugeordnet', inplace=True)


In [ ]:
#zeigt, dass wir für jedes Bundesland für jedes Jahr Werte vorliegen haben
pflichtarbeitsplätze['Bundesland'].value_counts()


In [ ]:
#kontrolle
pflichtarbeitsplätze['unbesetzt'].loc[pflichtarbeitsplätze.index][33] 



In [ ]:
pflichtarbeitsplätze.head(85)

In [ ]:
#dieser Wert war wirklich eine 0 und muss beibehalten werden
pflichtarbeitsplätze.at[33,'unbesetzt']= 0


In [ ]:
#kontrolle
pflichtarbeitsplätze['unbesetzt'].loc[pflichtarbeitsplätze.index][33] 


In [ ]:
# Funktion, die alle Indizes eines bestimmten Jahres auswählt und diese als Liste ausgibt
def get_year_index(jahr):
    i = pflichtarbeitsplätze.index
    index = pflichtarbeitsplätze["Jahr"] == jahr
    result = i[index]
    year_indices = result.tolist()
    return year_indices

In [ ]:
# Entfernen aller nicht-verfügbaren Daten
pflichtarbeitsplätze = pflichtarbeitsplätze.dropna()

In [ ]:
pflichtarbeitsplätze.head(85)

In [ ]:
# Anlegen einer Liste mit allen im Datensatz enthaltenen Jahren
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

# Anlegen zweier leerer Listen, die anschließend mit den Durchschnittswerten von privaten/öffentlichen Arbeitgebern befüllt werden.
durchschnittswerte_privat = []
durchschnittswerte_oeffentlich = []

# Jedes Jahr aus der year-Liste wird abgeglichen mit den Jahren aus dem Datensatz, um die Indizes jedes Jahres zu erhalten.
for year in years:
    get_year_index(year)
    quotenwerte = []

# An den ermittelten Index-Stellen wird geprüft, ob es sich beim Eintrag um einen privaten Arbeitgeber handelt.
# Ist das der Fall, wird der erreichte Quotenwert (aus der Spalte "Ist%") 
    for item in get_year_index(year):
        if pflichtarbeitsplätze["Arbeitgeber"][item] == "Privat":
            quotenwerte.append(pflichtarbeitsplätze["Ist%"][item])

# Die eben ermittelten Werte sind noch nach einzelnen Bundesländern getrennt. Nun soll daraus der Durchschnitt errechnet werden.
    gesamt = sum(quotenwerte)
    durchschnitt_jahr = gesamt/ 16
    durchschnittswerte_privat.append(durchschnitt_jahr)

# Das eben beschriebene Vorgehen wird in gleicher Weise noch einmal durchgeführt - dieses Mal für die öffentlichen Arbeitgeber.
for year in years:
    get_year_index(year)
    quotenwerte = []
    
    for item in get_year_index(year):
        if pflichtarbeitsplätze["Arbeitgeber"][item] == "Oeffentlich":
            quotenwerte.append(pflichtarbeitsplätze["Ist%"][item])
    
    gesamt = sum(quotenwerte)
    durchschnitt_jahr = gesamt/ 16
    durchschnittswerte_oeffentlich.append(durchschnitt_jahr)

In [ ]:
# Erstellung zweier getrennter Datensätze, die jeweils die erreichte Quote der privaten/öffentlichen Arbeitgeber enthalten sollen.
quote_privat = pd.DataFrame(years)
quote_privat.columns = ["Jahr"]
quote_privat['Arbeitgeber'] = "Privat"

quote_oeffentlich = pd.DataFrame(years)
quote_oeffentlich.columns = ["Jahr"]
quote_oeffentlich['Arbeitgeber'] = "Oeffentlich"

# Erstellung eines dritten Datensatzes, der für jedes Jahr die vorgegebene Quote von 5% enthalten soll und später im
# Schaubild eine gerade Linie als Orientierung zum Vergleich der anderen Werte darstellt.
quote = pd.DataFrame(years)
quote.columns = ["Jahr"]
quote['Arbeitgeber'] = "5%-Quote"

# Jedem Datensatz wird eine dritte Spalte hinzugefügt, die die tatsächlich erreichten Durchschnittswerte für jedes Jahr enthält. 
quote_privat['Quote'] = durchschnittswerte_privat
quote_oeffentlich['Quote'] = durchschnittswerte_oeffentlich
quote['Quote'] = 0.05

In [ ]:
quote_privat

In [ ]:
quote_oeffentlich

In [ ]:
# Zusammenfügen und Sortieren der drei einzelnen Datensätze
frames = [quote_privat, quote_oeffentlich, quote]
result = pd.concat(frames, ignore_index=True)
sorted_result = result.sort_values(by=['Jahr'], ascending=True)
sorted_result.reset_index(drop = True, inplace = True)

# Werte der Spalte "Quote" werden mit 100 multipliziert, um die tatsächlichen Prozentwerte anzugeben.
sorted_result['Quote'] = sorted_result['Quote'].apply(lambda x: x*100)

sorted_result

In [ ]:
# Erstellen der Visualisierung mit plotly.express
graph_quote = px.line(sorted_result,
                      x="Jahr",
                      y="Quote",
                      color='Arbeitgeber',
                      )

# Festlegen der y-Achse auf die range 0 bis 7, um die Daten im richtigen Verhältnis darzustellen
graph_quote.update_layout(yaxis_range=[0,7])

graph_quote.show()

In [ ]:
# Zusammenfügen aller erstellten Visualisierungen mit dash und Festlegen eines Standard-Stylesheets
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Konfiguration der Struktur der Seite mit Hilfe von dash_bootstrap_components mit html-Elementen und
# einfachen CSS-Befehlen mit "style={}"
app.layout=html.Div([
        dbc.Row(
            dbc.Col(
            html.Div([
                html.H1('Einleitung', style={'font-family': 'sans-serif', 'margin-top': 30}),
                html.P('Bereits im Jahr 2009 haben die Vereinten Nationen (UN) Deutschland nahegelegt, Werkstätten für Menschen mit Behinderungen (WfbM), wie sie in ihrer derzeitigen Form bestehen, abzuschaffen. Mehr als ein Jahrzehnt später gibt es sie allerdings immer noch und Themen wie Mindestlohn in Werkstätten und Integration in den allgemeinen Arbeitsmarkt werden weiterhin diskutiert. Im Rahmen eines datenjournalistischen Projekts haben die Masterstudierenden Jonas Armbruster, Margareta Holzreiter, Dariush Movahedian und Friederike Streib sich mit Daten zu diesen Themen befasst.', style={'font-family': 'sans-serif', 'margin-top': 30})
                    ]),
                width={"size": 10},
                   ), justify="center"
               ),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.H1('Wie haben sich die Entgelte in den Werkstätten entwickelt?', style={'font-family': 'sans-serif'}),
                    html.Div([
                        html.P('2022 ist der Mindestlohn in Deutschland auf 12 Euro angestiegen. Nicht allerdings für Werkstattbeschäftigte, denn sie befinden sich nur in einem „arbeitnehmerähnlichen Verhältnis“ und haben so keinen Anspruch auf den gesetzlichen Mindestlohn. Stattdessen erhalten sie für ihre Arbeit in einer Werkstatt ein Arbeitsentgelt, welches sich aus Arbeitsförderungsgeld (AFöG), Grundbetrag und einem leistungsabhängigen Steigerungsbetrag zusammensetzt. Werkstattbeschäftigte erhalten dieses Arbeitsentgelt meist zusätzlich zu einer Grundsicherung vom Amt oder einer Erwerbsminderungsrente.', style={'font-family': 'sans-serif', 'margin-top': 30}),
                        html.P('Das Entgelt in Sachsen liegt 2021 bei 167 Euro pro Monat und somit deutlich unter dem bundesweiten Durchschnitt von 215 Euro. Besonders groß war die Differenz im Jahr 2017: Hier betrug das Entgelt in Sachsen 107 Euro und damit nur knapp mehr als die Hälfte des Bundesdurchschnitts von etwa 207 Euro. Allerdings verzeichnete Sachsen von 2018 auf 2019 einen starken Anstieg von 114 auf 163 Euro. 2020 sank das durchschnittliche Entgelt in Sachsen allerdings noch einmal auf 154 Euro, als ursächlich werden pandemiebedingte Einschränkungen des Werkstattbetriebs angesehen.', style={'font-family': 'sans-serif'}),
                        dcc.Graph(id= 'line-graph1', figure=graph_entgelte)
                            ])
                        ]),
                    width={"size": 10},
                   ), justify="center"
               ),
# Text und Karte werden in einer Zeile nebeneinandergestellt, um für eine bessere Übericht und
# abwechslungsreichere Darstellung zu sorgen.
        dbc.Row([
            dbc.Col(
                html.Div([
                    html.P('In der nebenstehenden Karte ist die Höhe der Monatsgehälter in WfbM für alle Bundesländer abgetragen. Der Vergleich der Entwicklung der Entgelte zeigt allgemein einen Trend nach oben. Mit Start der Erhebung in Jahr 2009 liegt das höchste ausgezahlte Entgelt (hier in Bremen) bei 214 Euro. Bis zum Jahr 2021 wurde das Niveau der Entgelte kontinuierlich angehoben, sodass der Höchstwert nun bei 269 Euro liegt. Hamburg hat somit Bremen überholt und zahlt momentan die meisten Gelder an Beschäftigte in WfbM aus.', style={'font-family': 'sans-serif', 'margin-top': 40}),
                    html.P('Deutlich wird aus der Betrachtung der Karte ebenfalls, dass ein signifikanter Unterschied zwischen den alten und neuen Bundesländern besteht. Während im Jahr 2009 Hessen mit 153 Euro den niedrigsten Wert unter den alten Bundesländern auszahlt, kommt Thüringen mit dem höchsten Wert von 145 Euro für die neuen Bundesländer gerade so auf ein ähnliches Niveau. Dieser Unterschied hält bis ins Jahr 2021 an, wobei besonders Sachsen mit dem konstant niedrigsten Entgelt auffällt. Der Wechsel von 2016 auf 2017 stellt für beinahe alle Bundesländer den höchsten Anstieg der Entgelte dar. Brandenburg ist in 2017 ausgeblendet, da für dieses Jahr keine Werte aus dem Bundesland vorliegen.', style={'font-family': 'sans-serif'}),
                        ]),
                    width={"size": 5},
                   ),
            dbc.Col(
                html.Div([
                    dcc.Graph(id= 'map-graph1', figure=karte_entgelte)
                        ]),
                width={"size": 5}
                   )
               ],justify="center"),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.H1('Wie unterscheidet sich die Erfüllung der Pflichtarbeitsplatzquote bei öffentlichen und privaten Arbeitgebern?', style={'font-family': 'sans-serif', 'margin-top': 50}),
                    html.Div([
                        html.P('Um die Inklusion voranzutreiben, hat die Bundesregierung eine sog. Pflichtarbeitsplatzquote von 5% für öffentliche und private Arbeitgeber erlassen. Sie soll die Beschäftigung von schwerbehinderten Menschen auf dem ersten Arbeitsmarkt erhöhen. Bei Nichteinhaltung müssen Arbeitgeber (Betriebe ab 20 Mitarbeiter*innen) eine Ausgleichsabgabe zahlen.', style={'font-family': 'sans-serif', 'margin-top': 30}),
                        html.P('Wie man der nachfolgenden Grafik entnehmen kann, erreichen nur die öffentlichen Arbeitgeber die Quote. Seit 2009 liegt die öffentliche Seite zwischen 6 und 6,5%. Private Arbeitgeber liegen unter der Pflichtarbeitsquote. 2019 kam es zu einem leichten Anstieg bis zum Jahr 2020 auf 4%. Es ist davon auszugehen, dass die Quote weiterhin nicht erfüllt wird. Private Arbeitsgber müssen also die Ausgleichsabgabe bezahlen.', style={'font-family': 'sans-serif'}),
                        dcc.Graph(id= 'line-graph2', figure=graph_quote)
                            ])
                        ]),
                    width={"size": 10},
                    ), justify="center"
                ),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.H1('Wo in Sachsen gibt es anerkannte Werkstätten und bieten diese eine besondere Berufsförderung an?', style={'font-family': 'sans-serif'}),
                        ]),
                    width={"size": 10},
                    ), justify="center"
                ),        
        dbc.Row([
            dbc.Col(
                html.Div([
                    html.P('Wie bereits erwähnt, sind Werkstätten in Deutschland dazu verpflichtet, die Beschäftigten auf eine Teilnahme am Arbeitsleben auf dem ersten Arbeitsmarkt vorzubereiten. Grundlegend bieten alle Werkstätten Maßnahmen an, die dieses Ziel verfolgen. Es gibt allerdings einzelne Betriebe, die ihren Beschäftigten besondere Möglichkeiten zur Berufsförderung anbieten. Hierbei muss unterschieden werden zwischen Werkstätten, die Praktika auf dem ersten Arbeitsmarkt ermöglichen, Werkstätten, deren Beschäftigten Außenarbeitsplätze in Unternehmen des ersten Arbeitsmarkts zur Verfügung stehen, und Werkstätten, die beispielsweise durch besondere Vorbereitung und Begleitung beim Prozess des Wechsels auf den ersten Arbeitsmarkt unterstützen. Die nebenstehende Grafik zeigt, welche Werkstätten eine Förderung im Sinne der letzten Beschreibung anbieten.', style={'font-family': 'sans-serif', 'margin-top': 30}),
                        ]),
                    width={"size": 3},
                   ),
            dbc.Col(
                html.Div([
                    dcc.Graph(id= 'map-graph2', figure=karte_werkstätten)
                        ]),
                width={"size": 7}
                   )
               ],justify="center"),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.P('Zunächst ist grundlegend erkennbar, in welcher Weise sich die WfbM auf das Gebiet des Bundeslandes Sachsen aufteilen. Insgesamt zeigt sich eine recht gleichmäßige Verteilung mit einer Ballung in der Nähe größerer Städte, die aufgrund der Einwohnerzahl dieser Gebiete auch erwartbar ist. Im Gebiet zwischen Chemnitz und Dresden und der tschechischen Grenze hingegen, gibt es für die Größe des Gebiets verhältnismäßig wenige Werkstätten. Anzumerken ist hierbei, dass die Karte nicht alle Einzelwerkstätten Sachsens auflistet, sondern nur die Betriebe darstellt. In einigen Fällen gehören zu einem Betrieb mehrere Einzelwerkstätten, die voneinander getrennt sind. Dementsprechend zeigt die Karte 61 Punkte, wohingegen die Gesamtzahl aller einzelnen Werkstätten in Sachsen 116 beträgt. Unter den Werkstätten gibt es einige Einzelbetriebe, allerdings wird die Mehrzahl der Werkstätten von größeren Trägern wie der Diakonie, der Lebenshilfe oder dem Christlichen Sozialwerk geführt.', style={'font-family': 'sans-serif'}),
                    html.P('Bei der Betrachtung der Werkstätten, die eine besondere Berufsförderung anbieten, fällt auf, dass dies nicht einmal die Hälfte der Werkstätten in Sachsen umfasst – in Leipzig gibt es beispielsweise keine einzige Werkstatt mit besonderer Berufsförderung. Im Raum Dresden hingegen trifft dies auf den Großteil der dort ansässigen Werkstätten zu.', style={'font-family': 'sans-serif'}),
                        ]),
                    width={"size": 10},
                    ), justify="center"
                ),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.H1('Wie hat sich die Anzahl der anerkannten Werkstätten in Sachsen und ganz Deutschland entwickelt?', style={'font-family': 'sans-serif', 'margin-top': 30}),
                    html.Div([
                        html.P('Werkstätten für Menschen mit Behinderung haben laut Werkstättenverordnung die Aufgabe, Menschen mit einer Beeinträchtigung die Teilhabe am Arbeitsleben zu ermöglichen. Zentrale Aufgabe ist es auch, die Beschäftigten durch geeignete Maßnahmen auf eine Arbeit auf dem ersten Arbeitsmarkt vorzubereiten.', style={'font-family': 'sans-serif', 'margin-top': 30}),
                        html.P('Im Jahr 2015 wurde erstmals die Umsetzung der UN-Behindertenrechtskonvention (UN-BRK) in Deutschland durch einen UN-Fachausschuss geprüft. Der Fachausschuss bemängelte unter anderem eine unzureichende Integration von Menschen mit Behinderung in den ersten Arbeitsmarkt und kritisierte die Segregation der Menschen in Behindertenwerkstätten. Deutschland wurde daraufhin aufgefordert, eine Ausstiegsstrategie aus dem System der Werkstätten zu entwickeln und den allgemeinen Arbeitsmarkt inklusiver zu gestalten.', style={'font-family': 'sans-serif'}),
                        html.P('Im Jahr 2009 gab es deutschlandweit 715 amtlich anerkannte Werkstätten für Menschen mit Behinderung. In den folgenden Jahren ist die Anzahl der Werkstätten weiter gestiegen und lag 2021 bei 736 Werkstätten in ganz Deutschland. Auch nach den Empfehlungen durch den UN-Ausschuss 2015 ist kein Einbruch erkennbar.', style={'font-family': 'sans-serif'}),
                        html.P('Der Freistaat Sachsen liegt bei der Anzahl der amtlich anerkannten Werkstätten über dem bundesweiten Durchschnitt. Während es 2021 pro Bundesland durchschnittlich 46 Werkstätten gab, sind es in Sachsen 60 Stück. Hier ist die Anzahl der anerkannten Werkstätten in den vergangenen 14 Jahren kontinuierlich gleich geblieben. Anzumerken ist allerdings, dass die Größe der Werkstätten variiert und der Vergleich deshalb keine übergeordneten Aussagen zulässt.', style={'font-family': 'sans-serif'}),
                            ])
                        ]),
                    width={"size": 10},
                    ), justify="center"
                ),
        dbc.Row([
            dbc.Col(
                html.Div([
                    dcc.Graph(id= 'line-graph3', figure=graph_werkstätten)
                        ]),
                    width={"size": 5},
                   ),
            dbc.Col(
                html.Div([
                    dcc.Graph(id= 'line-graph4', figure=graph_werkstätten2)
                        ]),
                width={"size": 5}
                   )
               ],justify="center"),
        dbc.Row(
            dbc.Col(
                html.Div([
                    html.H1('Wie viele Menschen sind und waren in anerkannten Werkstätten in Sachsen und Gesamtdeutschland beschäftigt?', style={'font-family': 'sans-serif'}),
                    html.Div([
                        html.P('Das Recht auf einen Platz in einer Werkstatt für Menschen mit Behinderung hat, wer als voll erwerbsgemindert eingestuft ist. Weitere Voraussetzungen für eine Beschäftigung im Arbeitsbereich einer Werkstatt sind, dass die behinderten Beschäftigten ein Mindestmaß an wirtschaftlich verwertbarer Arbeit leisten können und keinen außerordentlichen Pflegebedarf haben.', style={'font-family': 'sans-serif', 'margin-top': 30}),
                        html.P('Seit der Unterzeichnung der UN-BRK 2009 ist die Anzahl der belegten Plätze in Mitgliedswerkstätten der Bundesarbeitsgemeinschaft WfbM (BAG WfbM) in Deutschland von 265.178 leicht auf 293.069 (2021) gestiegen. Bei den belegten Plätzen liegt Sachsen hier unter dem Bundesdurchschnitt und weist seit 2012 einen Rückgang der belegten Plätze auf. Während es 2012 einen Peak mit 13.067 belegten Plätzen gab, sind in Sachsen 9 Jahre später nur noch 10.449 Plätze in den BAG-Mitgliedswerkstätten belegt gewesen.', style={'font-family': 'sans-serif'}),
                        dcc.Graph(id= 'line-graph5', figure=graph_werkstattplätze_durchschnitt)
                            ])
                        ]),
                    width={"size": 10},
                    ), justify="center"
                )
                    ])


app.run_server(debug=False, port=8010, host='localhost')
